# Extraindo empresas

## Extraindo todas empresas do Brazil da categoria BESST

In [20]:
import requests
import yfinance as yf
import pandas as pd
import os

# --- PASSO 1: CONFIGURAÇÃO DOS SETORES-ALVO ---
# A busca é focada nestes setores.
setores_alvo = [
    "Finance",
    "Utilities",
    "Communications",
    "Industrial Services"
]

# --- FUNÇÃO PRINCIPAL COM A LÓGICA FINAL ---
def scanner_final_com_tipo(setores):
    """
    Busca AÇÕES (stock) e FUNDOS (fund) dos setores-alvo, filtrando por 
    país e incluindo a coluna 'Tipo' no resultado.
    """
    print("Iniciando scanner final focado (filtrando por tipo 'stock' e 'fund')...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    # Etapa 1: Mapear tickers, setores, logos e tipos da brapi
    print("Mapeando ativos dos setores alvo...")
    tickers_para_analisar = {}
    for stock in lista_completa_brapi:
        setor_brapi = stock.get('sector')
        tipo_ativo = stock.get('type')

        # <<< CONDIÇÃO ATUALIZADA PARA INCLUIR 'FUND' >>>
        # Agora o script verifica o setor E se o tipo é 'stock' OU 'fund'
        if setor_brapi in setores and tipo_ativo in ['stock', 'fund']:
            ticker = f"{stock.get('stock')}.SA"
            
            # Armazenamos todos os dados que vêm da brapi
            tickers_para_analisar[ticker] = {
                'setor': setor_brapi,
                'logo': stock.get('logo'),
                'tipo': tipo_ativo 
            }
    
    total = len(tickers_para_analisar)
    print(f"✅ Mapeamento concluído. {total} ativos dos tipos 'stock' e 'fund' serão analisados.")

    # Etapa 2: Coleta de Dados Detalhada
    dados_finais = {}
    print("\nIniciando coleta detalhada de dados no yfinance...")
    for i, (ticker, brapi_data) in enumerate(tickers_para_analisar.items()):
        try:
            print(f"Analisando [{i+1}/{total}]: {ticker}...")
            stock = yf.Ticker(ticker)
            info = stock.info
            
            if info.get("country") != "Brazil":
                print(f"  -> DESCARTADO: {ticker} não é do Brasil.")
                continue
            
            # Coleta dos dados, incluindo a nova coluna 'Tipo'
            dados_finais[ticker] = {
                "Empresa": info.get("longName", "N/A"),
                "Setor (brapi)": brapi_data['setor'],
                "Subsetor (yfinance)": info.get("industry", "N/A"),
                "País": info.get("country", "N/A"),
                "Tipo": brapi_data['tipo'],
                "Market Cap": info.get("marketCap"),
                "Logo": brapi_data['logo'],
            }
            print(f"  -> ✅ INCLUÍDO: {ticker}")

        except Exception as e:
            print(f"  -> ERRO: Falha ao processar {ticker}. Erro: {e}")
            continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_final_com_tipo(setores_alvo)

if resultados:
    print("\n✅ Análise finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(
             lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and x > 0 else "N/A"
         )

    df_ordenado = df_resultado.sort_values(by=['Setor (brapi)', 'Market Cap Num'], ascending=[True, False])
    
    colunas_finais = ['Ticker', 'Empresa', 'Setor (brapi)', 'Subsetor (yfinance)', 'País', 'Tipo', 'Market Cap', 'Logo']
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Final (Ações e Fundos) ---")
    print(df_final.to_string(index=False))

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        output_folder = r"E:\Github\finance-manager\datasets"
        # Nome do arquivo atualizado para refletir a inclusão de fundos
        output_filename = "scanner_acoes_e_fundos_filtrado.xlsx" 
        full_path = os.path.join(output_folder, output_filename)
        
        os.makedirs(output_folder, exist_ok=True)
        
        df_final.to_excel(full_path, index=False)
        print(f"\n✅ Resultados salvos com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

Iniciando scanner final focado (filtrando por tipo 'stock' e 'fund')...
Mapeando ativos dos setores alvo...
✅ Mapeamento concluído. 304 ativos dos tipos 'stock' e 'fund' serão analisados.

Iniciando coleta detalhada de dados no yfinance...
Analisando [1/304]: BBDC4.SA...
  -> ✅ INCLUÍDO: BBDC4.SA
Analisando [2/304]: BBAS3.SA...
  -> ✅ INCLUÍDO: BBAS3.SA
Analisando [3/304]: VAMO3.SA...
  -> ✅ INCLUÍDO: VAMO3.SA
Analisando [4/304]: B3SA3.SA...
  -> ✅ INCLUÍDO: B3SA3.SA
Analisando [5/304]: ITSA4.SA...
  -> ✅ INCLUÍDO: ITSA4.SA
Analisando [6/304]: CSAN3.SA...
  -> ✅ INCLUÍDO: CSAN3.SA
Analisando [7/304]: CPLE6.SA...
  -> ✅ INCLUÍDO: CPLE6.SA
Analisando [8/304]: RENT3.SA...
  -> ✅ INCLUÍDO: RENT3.SA
Analisando [9/304]: ITUB4.SA...
  -> ✅ INCLUÍDO: ITUB4.SA
Analisando [10/304]: MOVI3.SA...
  -> ✅ INCLUÍDO: MOVI3.SA
Analisando [11/304]: CMIG4.SA...
  -> ✅ INCLUÍDO: CMIG4.SA
Analisando [12/304]: CPLE3.SA...
  -> ✅ INCLUÍDO: CPLE3.SA
Analisando [13/304]: BBDC3.SA...
  -> ✅ INCLUÍDO: BBDC3.SA
An

In [ ]:
# import pandas as pd

# df_dados2 = pd.read_excel(r'E:\Github\finance-manager\datasets\scanner_acoes_filtrado.xlsx')

# print(df_dados2)

# Transformação

## Relatorio completo

In [2]:
import yfinance as yf
import pandas as pd
import os # Importando a biblioteca para lidar com pastas e caminhos
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# --- PASSO 1: LER A LISTA DE ATIVOS DO SEU ARQUIVO ---
# O caminho do arquivo de entrada permanece o mesmo.
arquivo_input = r"E:\Github\finance-manager\datasets\scanner_acoes_e_fundos_filtrado.xlsx"

try:
    print(f"Lendo a lista de ativos do arquivo '{arquivo_input}'...")
    df_input = pd.read_excel(arquivo_input)
    
    # Adicionando 'Tipo' à lista de colunas esperadas na verificação de segurança.
    colunas_esperadas = ['Ticker', 'Empresa', 'Setor (brapi)', 'País', 'Tipo', 'Market Cap', 'Logo']
    colunas_faltantes = [col for col in colunas_esperadas if col not in df_input.columns]
    
    if colunas_faltantes:
        print("\n" + "="*50)
        print("⚠️ AVISO: Colunas Faltantes no Arquivo de Entrada!")
        print(f"O arquivo '{arquivo_input}' não possui as seguintes colunas: {colunas_faltantes}")
        print("Verifique se você está usando o arquivo correto gerado pelo script anterior.")
        print("="*50 + "\n")

    tickers_para_analisar = df_input['Ticker'].tolist()
    print(f"✅ Sucesso. {len(tickers_para_analisar)} ativos serão analisados.")

except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{arquivo_input}' não foi encontrado.")
    exit()
except Exception as e:
    print(f"❌ ERRO: Ocorreu um problema ao ler o arquivo: {e}")
    exit()

# --- PASSO 2: BUSCAR OS DADOS DETALHADOS PARA CADA TICKER DA LISTA ---
dados_detalhados_acoes = {}
print("\nIniciando a coleta de dados detalhados...")

for ticker in tickers_para_analisar:
    try:
        print(f"Analisando: {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info
        
        info_original = df_input[df_input['Ticker'] == ticker].iloc[0]

        # --- Cálculo da Variação de 5 Anos ---
        variacao_5a = "N/A"
        try:
            end_date = date.today()
            start_date = end_date - relativedelta(years=5)
            hist_5a = stock.history(start=start_date, end=end_date)
            if not hist_5a.empty and len(hist_5a) > 1:
                preco_inicial = hist_5a['Close'].iloc[0]
                preco_final = hist_5a['Close'].iloc[-1]
                if preco_inicial > 0:
                    variacao_5a = ((preco_final / preco_inicial) - 1) * 100
        except Exception:
            variacao_5a = "N/A"

        # Coletando os dados, incluindo a nova coluna 'Tipo'
        dados_detalhados_acoes[ticker] = {
            "Empresa": info_original.get('Empresa', 'N/A'),
            "Setor (brapi)": info_original.get('Setor (brapi)', 'N/A'), 
            "País": info_original.get('País', 'N/A'),
            # <<< NOVA COLUNA 'TIPO' ADICIONADA AQUI >>>
            "Tipo": info_original.get('Tipo', 'N/A'),
            "Market Cap (R$)": info_original.get('Market Cap', 'N/A'),
            "Logo": info_original.get('Logo', 'N/A'),
            
            # Dados do yfinance
            "Data Cotação": datetime.fromtimestamp(info.get("regularMarketTime", 0)).strftime('%Y-%m-%d %H:%M:%S'),
            "Preço Atual": info.get("currentPrice", 0),
            "Preço Fechamento": info.get("previousClose", 0),
            "Volume Negociado": info.get("volume", 0),
            "P/L": info.get("trailingPE", "N/A"),
            "P/VP": info.get("priceToBook", "N/A"),
            "Dividend Yield (%)": info.get("dividendYield", 0) * 100,
            "DY 5 Anos Média (%)": info.get("fiveYearAvgDividendYield", "N/A"),
            "Dividendos 12m (R$)": info.get("trailingAnnualDividendRate", "N/A"),
            "ROE (%)": info.get("returnOnEquity", 0) * 100,
            "Margem Líquida (%)": info.get("profitMargins", 0) * 100,
            "Dívida/EBITDA": info.get("debtToEbitda", "N/A"),
            "Variação 12m (%)": info.get("52WeekChange", 0) * 100,
            "Variação 5a (%)": variacao_5a,
        }
        print(f"✅ Dados de {ticker} obtidos com sucesso.")

    except IndexError:
        print(f"❌ ERRO: O ticker '{ticker}' está na sua lista, mas não foi encontrado no arquivo Excel. Pulando.")
    except Exception as e:
        print(f"❌ Não foi possível obter dados para {ticker}. Erro: {e}")

print("\nColeta de dados finalizada.")

# --- PASSO 3: ORGANIZAR E SALVAR O NOVO RELATÓRIO DETALHADO ---
if dados_detalhados_acoes:
    df_resultado_detalhado = pd.DataFrame.from_dict(dados_detalhados_acoes, orient="index")

    colunas_para_formatar = [
        "Preço Atual", "Preço Fechamento", "P/L", "P/VP", "Dividend Yield (%)",
        "DY 5 Anos Média (%)", "Dividendos 12m (R$)", "ROE (%)",
        "Margem Líquida (%)", "Variação 12m (%)", "Variação 5a (%)"
    ]
    for col in colunas_para_formatar:
        if col in df_resultado_detalhado.columns:
            df_resultado_detalhado[col] = df_resultado_detalhado[col].apply(
                lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x
            )

    print("\n--- Tabela Comparativa de Indicadores Fundamentalistas ---")
    print(df_resultado_detalhado.to_string())

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        # Define o caminho da pasta e o nome do arquivo
        output_folder = r"E:\Github\finance-manager\datasets"
        output_filename = "relatorio_completo_b3.xlsx"
        full_path = os.path.join(output_folder, output_filename)
        
        # Cria a pasta de destino se ela não existir
        os.makedirs(output_folder, exist_ok=True)
        
        # Salva o arquivo no caminho completo, substituindo se existir
        df_resultado_detalhado.to_excel(full_path)
        print(f"\n✅ Relatório detalhado salvo com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhum dado foi coletado.")

Lendo a lista de ativos do arquivo 'E:\Github\finance-manager\datasets\scanner_acoes_e_fundos_filtrado.xlsx'...
✅ Sucesso. 146 ativos serão analisados.

Iniciando a coleta de dados detalhados...
Analisando: VIVT3.SA...
✅ Dados de VIVT3.SA obtidos com sucesso.
Analisando: TIMS3.SA...
✅ Dados de TIMS3.SA obtidos com sucesso.
Analisando: FIQE3.SA...
✅ Dados de FIQE3.SA obtidos com sucesso.
Analisando: BRST3.SA...
✅ Dados de BRST3.SA obtidos com sucesso.
Analisando: TELB3.SA...
✅ Dados de TELB3.SA obtidos com sucesso.
Analisando: TELB4.SA...
✅ Dados de TELB4.SA obtidos com sucesso.
Analisando: DESK3.SA...
✅ Dados de DESK3.SA obtidos com sucesso.
Analisando: OIBR3.SA...
✅ Dados de OIBR3.SA obtidos com sucesso.
Analisando: OIBR4.SA...
✅ Dados de OIBR4.SA obtidos com sucesso.
Analisando: DTCY3.SA...
✅ Dados de DTCY3.SA obtidos com sucesso.
Analisando: ITUB4.SA...
✅ Dados de ITUB4.SA obtidos com sucesso.
Analisando: ITUB3.SA...
✅ Dados de ITUB3.SA obtidos com sucesso.
Analisando: BPAC3.SA...
✅

# Analises

In [9]:
import pandas as pd

df_dados = pd.read_excel(r'E:\Github\finance-manager\datasets\relatorio_completo_b3.xlsx')

print(df_dados.head(2))

  Unnamed: 0                 Empresa   Setor (brapi)    País   Tipo  \
0   VIVT3.SA  Telefônica Brasil S.A.  Communications  Brazil  stock   
1   TIMS3.SA                TIM S.A.  Communications  Brazil  stock   

  Market Cap (R$)                                     Logo  \
0     R$ 98.50 Bi  https://icons.brapi.dev/icons/VIVT3.svg   
1     R$ 51.57 Bi  https://icons.brapi.dev/icons/TIMS3.svg   

          Data Cotação  Preço Atual  Preço Fechamento  ...    P/L  P/VP  \
0  2025-06-27 17:07:37        30.40             30.48  ...  17.47  1.45   
1  2025-06-27 17:07:33        21.33             21.67  ...  15.02  2.11   

   Dividend Yield (%)  DY 5 Anos Média (%)  Dividendos 12m (R$)  ROE (%)  \
0               378.0                 5.39                 0.74     8.36   
1               589.0                 4.84                 0.80    13.87   

   Margem Líquida (%)  Dívida/EBITDA  Variação 12m (%)  Variação 5a (%)  
0               10.07            NaN             32.49            34.8

In [ ]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# --- PASSO 1: LER A LISTA DE ATIVOS DO SEU ARQUIVO ---
arquivo_input = r"E:\Github\finance-manager\datasets\scanner_acoes_e_fundos_filtrado.xlsx"

try:
    print(f"Lendo a lista de ativos do arquivo '{arquivo_input}'...")
    df_input = pd.read_excel(arquivo_input)
    colunas_esperadas = ['Ticker', 'Empresa', 'Setor (brapi)', 'País', 'Tipo', 'Market Cap', 'Logo']
    colunas_faltantes = [col for col in colunas_esperadas if col not in df_input.columns]
    if colunas_faltantes:
        print(f"\n⚠️ AVISO: Colunas Faltantes no Arquivo de Entrada: {colunas_faltantes}\n")

    tickers_para_analisar = df_input['Ticker'].tolist()
    print(f"✅ Sucesso. {len(tickers_para_analisar)} ativos serão analisados.")

except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{arquivo_input}' não foi encontrado.")
    exit()
except Exception as e:
    print(f"❌ ERRO: Ocorreu um problema ao ler o arquivo: {e}")
    exit()

# --- PASSO 2: BUSCAR OS DADOS DETALHADOS COM LÓGICA CONDICIONAL ---
dados_detalhados_acoes = {}
print("\nIniciando a coleta de dados detalhados com análise especialista...")

for ticker in tickers_para_analisar:
    try:
        print(f"Analisando: {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info
        
        info_original = df_input[df_input['Ticker'] == ticker].iloc[0]
        setor_principal = info_original.get('Setor (brapi)', 'N/A')

        # --- DADOS COMUNS A TODOS OS SETORES ---
        dy_pela_taxa = "N/A"
        try:
            taxa_anual_div = info.get("trailingAnnualDividendRate")
            preco_atual = info.get("currentPrice")
            if taxa_anual_div is not None and preco_atual is not None and preco_atual > 0:
                dy_pela_taxa = (taxa_anual_div / preco_atual) * 100
        except Exception:
            dy_pela_taxa = "N/A"

        dados_base = {
            "Empresa": info_original.get('Empresa', 'N/A'),
            "Setor (brapi)": setor_principal, 
            "Tipo": info_original.get('Tipo', 'N/A'),
            "Market Cap (R$)": info_original.get('Market Cap', 'N/A'),
            "Logo": info_original.get('Logo', 'N/A'),
            "Preço Atual": info.get("currentPrice", 0),
            "P/L": info.get("trailingPE", "N/A"),
            "P/VP": info.get("priceToBook", "N/A"),
            "DY (Taxa 12m, %)": dy_pela_taxa,
            "DY 5 Anos Média (%)": info.get("fiveYearAvgDividendYield", "N/A"),
            "Último Dividendo (R$)": info.get("lastDividendValue", "N/A"),
            "Data Últ. Div.": pd.to_datetime(info.get("lastDividendDate"), unit='s') if info.get("lastDividendDate") else "N/A",
            "Data Ex-Div.": pd.to_datetime(info.get("exDividendDate"), unit='s') if info.get("exDividendDate") else "N/A",
            "Payout Ratio (%)": info.get("payoutRatio", 0) * 100,
        }
        
        # <<< LÓGICA CONDICIONAL "ESPECIALISTA" REINTRODUZIDA AQUI >>>
        if setor_principal == 'Finance':
            print("  -> Setor 'Finance' detectado. Métricas de dívida industrial não aplicáveis.")
            dados_especificos = {
                "ROE (%)": info.get("returnOnEquity", 0) * 100,
                "Dívida Total": "N/A",      # Ignorado propositalmente
                "Dívida/EBITDA": "N/A",     # Ignorado propositalmente
            }
        else:
            print("  -> Setor Industrial/Serviços detectado. Coletando métricas de dívida.")
            total_debt = info.get('totalDebt', None)
            ebitda = info.get('ebitda', None)
            divida_ebitda = "N/A"
            if total_debt is not None and ebitda is not None and ebitda != 0:
                divida_ebitda = total_debt / ebitda

            dados_especificos = {
                "ROE (%)": info.get("returnOnEquity", 0) * 100,
                "Dívida Total": total_debt,
                "Dívida/EBITDA": divida_ebitda,
            }
        
        # Unindo os dicionários de dados
        dados_detalhados_acoes[ticker] = {**dados_base, **dados_especificos}
        print(f"✅ Dados de {ticker} obtidos com sucesso.")

    except Exception as e:
        print(f"❌ Não foi possível obter dados para {ticker}. Erro: {e}")

# --- PASSO 3: ORGANIZAR E SALVAR ---
if dados_detalhados_acoes:
    df_resultado_detalhado = pd.DataFrame.from_dict(dados_detalhados_acoes, orient="index")
    
    # <<< FORMATAÇÃO DA 'DÍVIDA TOTAL' REINTRODUZIDA >>>
    if "Dívida Total" in df_resultado_detalhado.columns:
        df_resultado_detalhado["Dívida Total"] = df_resultado_detalhado["Dívida Total"].apply(
            lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and isinstance(x, (int, float)) and x != 0 else "N/A"
        )
    
    colunas_numericas = ["Preço Atual", "P/L", "P/VP"]
    for col in colunas_numericas:
        if col in df_resultado_detalhado.columns:
            df_resultado_detalhado[col] = df_resultado_detalhado[col].apply(
                lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x
            )
            
    colunas_percentual = ["DY (Taxa 12m, %)", "DY 5 Anos Média (%)", "ROE (%)", "Payout Ratio (%)"]
    for col in colunas_percentual:
        if col in df_resultado_detalhado.columns:
            df_resultado_detalhado[col] = df_resultado_detalhado[col].apply(
                lambda x: f"{x:.2f}%" if isinstance(x, (int, float)) else x
            )

    colunas_moeda = ["Último Dividendo (R$)"]
    for col in colunas_moeda:
         if col in df_resultado_detalhado.columns:
            df_resultado_detalhado[col] = df_resultado_detalhado[col].apply(
                lambda x: f"R$ {x:.2f}" if isinstance(x, (int, float)) else x
            )
    
    colunas_data = ["Data Últ. Div.", "Data Ex-Div."]
    for col in colunas_data:
        if col in df_resultado_detalhado.columns:
            df_resultado_detalhado[col] = pd.to_datetime(df_resultado_detalhado[col], errors='coerce').dt.strftime('%d-%m-%Y')

    print("\n--- Tabela Comparativa de Indicadores (Análise Especialista) ---")
    print(df_resultado_detalhado.to_string())

    try:
        output_folder = r"E:\Github\finance-manager\datasets"
        output_filename = "relatorio_analise_b3.xlsx"
        full_path = os.path.join(output_folder, output_filename)
        os.makedirs(output_folder, exist_ok=True)
        df_resultado_detalhado.to_excel(full_path)
        print(f"\n✅ Relatório detalhado salvo com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"❌ Erro ao salvar o novo arquivo Excel: {e}")
else:
    print("\nNenhum dado foi coletado.")

Lendo a lista de ativos do arquivo 'E:\Github\finance-manager\datasets\scanner_acoes_e_fundos_filtrado.xlsx'...
✅ Sucesso. 146 ativos serão analisados.

Iniciando a coleta de dados detalhados com análise especialista...
Analisando: VIVT3.SA...
  -> Setor Industrial/Serviços detectado. Coletando métricas de dívida.
✅ Dados de VIVT3.SA obtidos com sucesso.
Analisando: TIMS3.SA...
  -> Setor Industrial/Serviços detectado. Coletando métricas de dívida.
✅ Dados de TIMS3.SA obtidos com sucesso.
Analisando: FIQE3.SA...
  -> Setor Industrial/Serviços detectado. Coletando métricas de dívida.
✅ Dados de FIQE3.SA obtidos com sucesso.
Analisando: BRST3.SA...
  -> Setor Industrial/Serviços detectado. Coletando métricas de dívida.
✅ Dados de BRST3.SA obtidos com sucesso.
Analisando: TELB3.SA...
  -> Setor Industrial/Serviços detectado. Coletando métricas de dívida.
✅ Dados de TELB3.SA obtidos com sucesso.
Analisando: TELB4.SA...
  -> Setor Industrial/Serviços detectado. Coletando métricas de dívida.

Grok

In [ ]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import logging

# Configurar logging para registrar erros
logging.basicConfig(
    filename=r"E:\Github\finance-manager\datasets\erros_analise.log",
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# --- PASSO 1: LER A LISTA DE ATIVOS DO SEU ARQUIVO ---
arquivo_input = r"E:\Github\finance-manager\datasets\scanner_acoes_e_fundos_filtrado.xlsx"

try:
    print(f"Lendo a lista de ativos do arquivo '{arquivo_input}'...")
    df_input = pd.read_excel(arquivo_input)
    
    colunas_esperadas = ['Ticker', 'Empresa', 'Setor (brapi)', 'País', 'Tipo', 'Market Cap', 'Logo']
    colunas_faltantes = [col for col in colunas_esperadas if col not in df_input.columns]
    if colunas_faltantes:
        print(f"\n⚠️ AVISO: Colunas Faltantes no Arquivo de Entrada: {colunas_faltantes}\n")

    # Padronizar setores (remover espaços, converter para maiúsculas)
    df_input['Setor (brapi)'] = df_input['Setor (brapi)'].str.strip().str.upper()
    
    tickers_para_analisar = df_input['Ticker'].tolist()
    print(f"✅ Sucesso. {len(tickers_para_analisar)} ativos serão analisados.")

except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{arquivo_input}' não foi encontrado.")
    logging.error(f"Arquivo de entrada '{arquivo_input}' não encontrado.")
    exit()
except Exception as e:
    print(f"❌ ERRO: Ocorreu um problema ao ler o arquivo: {e}")
    logging.error(f"Erro ao ler o arquivo de entrada: {e}")
    exit()

# --- PASSO 2: BUSCAR OS DADOS DETALHADOS ---
dados_detalhados_acoes = {}
print("\nIniciando a coleta de dados detalhados...")

for ticker in tickers_para_analisar:
    try:
        print(f"Analisando: {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info
        info_original = df_input[df_input['Ticker'] == ticker].iloc[0]     

        # Cálculo de DY baseado na Taxa Anual
        dy_pela_taxa = 0.0
        try:
            taxa_anual_div = info.get("trailingAnnualDividendRate", 0)
            preco_atual = info.get("currentPrice", 0)
            if taxa_anual_div > 0 and preco_atual > 0:
                dy_pela_taxa = (taxa_anual_div / preco_atual) * 100
        except Exception as e:
            logging.error(f"Erro ao calcular DY para {ticker}: {e}")
            dy_pela_taxa = 0.0

        # Validar Market Cap
        market_cap = info_original.get('Market Cap', 0)
        if not isinstance(market_cap, (int, float)) or market_cap <= 0:
            market_cap = 0

        # Coletando os dados
        dados_detalhados_acoes[ticker] = {
            "Empresa": info_original.get('Empresa', 'N/A'),
            "Setor (brapi)": info_original.get('Setor (brapi)', 'N/A'),
            "Tipo": info_original.get('Tipo', 'N/A'),
            "Market Cap (R$)": market_cap,
            "Logo": info_original.get('Logo', 'N/A'),
            "Preço Atual": info.get("currentPrice", 0),
            "P/L": info.get("trailingPE", float('inf')) if info.get("trailingPE") and info.get("trailingPE") < 1000 else 0,
            "P/VP": info.get("priceToBook", float('inf')) if info.get("priceToBook") and info.get("priceToBook") < 1000 else 0,
            "ROE (%)": info.get("returnOnEquity", 0) * 100 if info.get("returnOnEquity") else 0,
            "Dívida Total": info.get("totalDebt", "N/A") if "FINANCEIRO" not in str(info_original.get('Setor (brapi)', '')).upper() else "N/A",
            # Painel de Dividendos Simplificado
            "DY (Taxa 12m, %)": dy_pela_taxa,
            "DY 5 Anos Média (%)": info.get("fiveYearAvgDividendYield", 0) if info.get("fiveYearAvgDividendYield") else 0,
            "Último Dividendo (R$)": info.get("lastDividendValue", 0) if info.get("lastDividendValue") else 0,
            "Data Últ. Div.": pd.to_datetime(info.get("lastDividendDate"), unit='s', errors='coerce') if info.get("lastDividendDate") else pd.NaT,
            "Data Ex-Div.": pd.to_datetime(info.get("exDividendDate"), unit='s', errors='coerce') if info.get("exDividendDate") else pd.NaT,
            "Payout Ratio (%)": info.get("payoutRatio", 0) * 100 if info.get("payoutRatio") else 0,
        }
        print(f"✅ Dados de {ticker} obtidos com sucesso.")

    except IndexError:
        print(f"❌ ERRO: O ticker '{ticker}' está na sua lista, mas não foi encontrado no arquivo Excel. Pulando.")
        logging.error(f"Ticker {ticker} não encontrado no arquivo Excel.")
    except Exception as e:
        print(f"❌ Não foi possível obter dados para {ticker}. Erro: {e}")
        logging.error(f"Erro ao obter dados para {ticker}: {e}")

print("\nColeta de dados finalizada.")

# --- PASSO 3: ORGANIZAR E SALVAR COM FORMATAÇÃO COMPLETA ---
if dados_detalhados_acoes:
    df_resultado_detalhado = pd.DataFrame.from_dict(dados_detalhados_acoes, orient="index")
    
    # Manter tipos numéricos durante o processamento
    colunas_numericas = ["Preço Atual", "P/L", "P/VP", "DY (Taxa 12m, %)", "DY 5 Anos Média (%)", "ROE (%)", "Payout Ratio (%)", "Último Dividendo (R$)", "Market Cap (R$)"]
    for col in colunas_numericas:
        if col in df_resultado_detalhado.columns:
            df_resultado_detalhado[col] = pd.to_numeric(df_resultado_detalhado[col], errors='coerce').fillna(0)

    # Formatação apenas para exibição e salvamento
    df_formatado = df_resultado_detalhado.copy()
    for col in ["Preço Atual", "P/L", "P/VP"]:
        if col in df_formatado.columns:
            df_formatado[col] = df_formatado[col].apply(
                lambda x: f"{x:.2f}" if x > 0 else "N/A"
            )
            
    for col in ["DY (Taxa 12m, %)", "DY 5 Anos Média (%)", "ROE (%)", "Payout Ratio (%)"]:
        if col in df_formatado.columns:
            df_formatado[col] = df_formatado[col].apply(
                lambda x: f"{x:.2f}%" if x > 0 else "N/A"
            )

    for col in ["Último Dividendo (R$)"]:
        if col in df_formatado.columns:
            df_formatado[col] = df_formatado[col].apply(
                lambda x: f"R$ {x:.2f}" if x > 0 else "N/A"
            )
    
    for col in ["Dívida Total"]:
        if col in df_formatado.columns:
            df_formatado[col] = df_formatado[col].apply(
                lambda x: f"R$ {x:.2f}" if isinstance(x, (int, float)) and x > 0 else "N/A"
            )

    colunas_data = ["Data Últ. Div.", "Data Ex-Div."]
    for col in colunas_data:
        if col in df_formatado.columns:
            df_formatado[col] = pd.to_datetime(df_formatado[col], errors='coerce').dt.strftime('%d-%m-%Y')

    print("\n--- Tabela Comparativa de Indicadores Fundamentalistas ---")
    print(df_formatado.to_string())

    try:
        output_folder = r"E:\Github\finance-manager\datasets"
        output_filename = "relatorio_analise_b3_2.xlsx"
        full_path = os.path.join(output_folder, output_filename)
        os.makedirs(output_folder, exist_ok=True)
        df_formatado.to_excel(full_path, index=True)
        print(f"\n✅ Relatório detalhado salvo com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"❌ Erro ao salvar o novo arquivo Excel: {e}")
        logging.error(f"Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhum dado foi coletado.")
    logging.warning("Nenhum dado foi coletado para os tickers fornecidos.")

Lendo a lista de ativos do arquivo 'E:\Github\finance-manager\datasets\scanner_acoes_e_fundos_filtrado.xlsx'...
✅ Sucesso. 146 ativos serão analisados.

Iniciando a coleta de dados detalhados...
Analisando: VIVT3.SA...
✅ Dados de VIVT3.SA obtidos com sucesso.
Analisando: TIMS3.SA...
✅ Dados de TIMS3.SA obtidos com sucesso.
Analisando: FIQE3.SA...
✅ Dados de FIQE3.SA obtidos com sucesso.
Analisando: BRST3.SA...
✅ Dados de BRST3.SA obtidos com sucesso.
Analisando: TELB3.SA...
✅ Dados de TELB3.SA obtidos com sucesso.
Analisando: TELB4.SA...
✅ Dados de TELB4.SA obtidos com sucesso.
Analisando: DESK3.SA...
✅ Dados de DESK3.SA obtidos com sucesso.
Analisando: OIBR3.SA...
✅ Dados de OIBR3.SA obtidos com sucesso.
Analisando: OIBR4.SA...
✅ Dados de OIBR4.SA obtidos com sucesso.
Analisando: DTCY3.SA...
✅ Dados de DTCY3.SA obtidos com sucesso.
Analisando: ITUB4.SA...
✅ Dados de ITUB4.SA obtidos com sucesso.
Analisando: ITUB3.SA...
✅ Dados de ITUB3.SA obtidos com sucesso.
Analisando: BPAC3.SA...
✅